#### Benchmarking Impersonation

- Voxceleb challenge framework can be somehow integrated for master voice testing
- Raw threshold as for now (similarly to INTERSPEECH 2019)
- Avg on scores, not on embeddings as for now (fot the sake of easiness)

In [40]:
from sklearn.metrics import roc_curve
import numpy as np
import pickle
import os

%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib

from IPython.display import display, HTML

import pandas as pd
pd.options.display.float_format = '{:,.4f}'.format

In [41]:
def save_obj(obj, name):
    with open(name, 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)
        
def load_obj(name):
    with open(name, 'rb') as f:
        return pickle.load(f)

### Parameters

In [70]:
nets = ['vggvox/v000', 'resnetse34l/v000', 'xvector/v001', 'resnet34vox/v001', 'resnet50vox/v001']

In [54]:
tars = [None, 1.0, 0.1]

In [55]:
pols = ['avg', 'any']

In [56]:
thrs_types = [None, 'avg', 'any']

### Benchmark ASV

In [57]:
def tuneThreshold(scores, labels, target_fa=None):
    far, tpr, thresholds = roc_curve(labels, scores, pos_label=1)
    frr = 1 - tpr
    frr = frr*100
    far = far*100
    if target_fa:
        idx = np.nanargmin(np.absolute((target_fa - far))) 
        return thresholds[idx], far[idx], frr[idx]
    idxE = np.nanargmin(np.absolute((frr - far)))
    eer  = max(far[idxE], frr[idxE])
    return thresholds[idxE], far[idxE], frr[idxE]

In [58]:
def count_far(targets, similarities, thr):
    fars = 0
    count = 0
    for t, s in zip(targets, similarities):
        if t == 0:
            if s >= thr:
                fars += 1
            count += 1
    return fars / count * 100

In [59]:
def count_frr(targets, similarities, thr):
    frrs = 0
    count = 0
    for t, s in zip(targets, similarities):
        if t == 1:
            if s < thr:
                frrs += 1
            count += 1
    return frrs / count * 100

In [60]:
vox1_test_results = {}
for net in nets:
    vox1_test_results[net] = pd.read_csv(os.path.join('../data/pt_models', net, 'test_vox1_sv_test.csv'))
    vox1_test_results[net] = vox1_test_results[net].loc[:, ~vox1_test_results[net].columns.str.contains('^Unnamed')]
    vox1_test_results[net].columns = ['score', 'label']

In [61]:
def groupScores(scores, labels, thrs_type, size=8):
    if thrs_type is None:
        return scores, labels
    func = np.mean if thrs_type == 'avg' else np.max
    grp_scores, grp_labels = [], []
    for i in range(0, len(scores), size):
        curr_scores = scores[i:i+size]
        grp_scores.append(func(curr_scores[1::2]))
        grp_labels.append(0)
        grp_scores.append(func(curr_scores[0::2]))
        grp_labels.append(1)
    return grp_scores, grp_labels

In [62]:
ress = {}
thrs = {}
for thrs_type in thrs_types:
    ress[thrs_type] = {}
    thrs[thrs_type] = {}
    for tar in tars:
        ress[thrs_type][tar] = {}
        thrs[thrs_type][tar] = {}
        for net in nets:
            if thrs_type is None:
                thr, far, frr = tuneThreshold(vox1_test_results[net]['score'].values, vox1_test_results[net]['label'].values, tar)
                thrs[thrs_type][tar][net] = thr
                ress[thrs_type][tar][net] = [np.mean([far, frr]), far, frr, thr, len(vox1_test_results[net].index)]
            else:
                grp_scores, grp_labels = groupScores(vox1_test_results[net]['score'].values, vox1_test_results[net]['label'].values, thrs_type)
                thr = thrs[None][tar][net]
                far = count_far(grp_labels, grp_scores, thr)
                frr = count_frr(grp_labels, grp_scores, thr)
                ress[thrs_type][tar][net] = [np.mean([far, frr]), far, frr, thr, len(vox1_test_results[net].index)]

In [63]:
for thrs_type in thrs_types:
    for tar in tars:
        tar_label = (thrs_type if thrs_type is not None else 'raw') + '  ' + ('FAR@'+str(tar) if tar is not None else 'EER')
        df = pd.DataFrame.from_dict(ress[thrs_type][tar], orient='index', columns=['eer', 'far', 'frr', 'thr', 'no-trials'])
        df.columns = pd.MultiIndex.from_tuples([(tar_label,'eer'), (tar_label,'far'), (tar_label,'frr'), (tar_label, 'thr'), (tar_label, 'no-trials')])
        df.style.set_properties(**{'width':'10em', 'text-align':'center'})
        display(HTML(df.to_html()))
        print()

### Benchmark MV

In [64]:
def computeImpersonation(fp, thr, pol, size=10):
    df = pd.read_csv(fp)
    imp_m, tot_m = 0, 0
    imp_f, tot_f = 0, 0
    user_ids_f, user_ids_m = [], []
    for i in range(0, len(df), size) : 
        user_id = i // size
        tot_f += 1 if df.loc[i, 'gender'] == 'f' else 0
        tot_m += 1 if df.loc[i, 'gender'] == 'm' else 0
        imp_r = len([i for i in df.loc[i:i+size-1, 'score'] if i >= thr]) if pol == 'any' else (1 if np.mean(df.loc[i:i+size-1, 'score']) > thr else 0)
        imp_f += 1 if df.loc[i, 'gender'] == 'f' and imp_r > 0 else 0
        imp_m += 1 if df.loc[i, 'gender'] == 'm' and imp_r > 0 else 0
        user_ids_f += [user_id] if df.loc[i, 'gender'] == 'f' and imp_r > 0 else []
        user_ids_m += [user_id] if df.loc[i, 'gender'] == 'm' and imp_r > 0 else []
    assert imp_m / tot_m <= 1.0 and imp_f / tot_f <= 1.0
    return imp_m, imp_f, user_ids_m, user_ids_f, tot_m, tot_f

In [65]:
mv_test_results = {}
for i1, pol in enumerate(pols):
    mv_test_results[pol] = {}
    for i2, tar in enumerate(tars):
        mv_test_results[pol][tar] = {}
        for i3, net in enumerate(nets):
            mv_test_results[pol][tar][net] = {}
            dp = os.path.join('../data/pt_models', net, 'mvcmp')
            for i4, fp in enumerate(os.listdir(dp)): 
                mvset = fp.split('__')[0]
                mvsam = fp.split('__')[1]
                if int(mvsam.split('.')[0].split('_')[-1]) <=5:
                    if mvset not in mv_test_results[pol][tar][net]:
                        mv_test_results[pol][tar][net][mvset] = {}
                    mv_test_results[pol][tar][net][mvset][mvsam] = computeImpersonation(os.path.join(dp, fp), thrs[None][tar][net], pol) 
                    print('>\r', pol, '(' + str(i1+1) + '/' + str(len(pols)) + ')', tar, '(' + str(i2+1) + '/' + str(len(tars)) + ')', 
                                 net, '(' + str(i3+1) + '/' + str(len(nets)) + ')',  fp, '('+str(i4+1)+'/'+str(len(os.listdir(dp))) +')', end='')
            save_obj(mv_test_results, 'test_vox2_mv_test')
            print(' > saved')

 avg (1/2) None (1/3) vggvox/v000 (1/5) gan_mf-f_sv__audio_original_female_5.csv (258/264) > saved
 avg (1/2) None (1/3) resnetse34l/v000 (2/5) real_f-f_mv__mv_01.csv (263/264) > saved
 avg (1/2) None (1/3) xvector/v001 (3/5) real_f-f_mv__mv_01.csv (263/264) > saved
 avg (1/2) None (1/3) resnet34vox/v001 (4/5) gan_mf-f_sv__audio_original_female_5.csv (258/264) > saved
 avg (1/2) None (1/3) resnet50vox/v001 (5/5) real_f-f_mv__mv_01.csv (263/264) > saved
 avg (1/2) 1.0 (2/3) vggvox/v000 (1/5) gan_mf-f_sv__audio_original_female_5.csv (258/264) > saved
 avg (1/2) 1.0 (2/3) resnetse34l/v000 (2/5) real_f-f_mv__mv_01.csv (263/264) > saved
 avg (1/2) 1.0 (2/3) xvector/v001 (3/5) real_f-f_mv__mv_01.csv (263/264) > saved
 avg (1/2) 1.0 (2/3) resnet34vox/v001 (4/5) gan_mf-f_sv__audio_original_female_5.csv (258/264) > saved
 avg (1/2) 1.0 (2/3) resnet50vox/v001 (5/5) real_f-f_mv__mv_01.csv (263/264) > saved
 avg (1/2) 0.1 (3/3) vggvox/v000 (1/5) gan_mf-f_sv__audio_original_female_5.csv (258/264) >

In [67]:
def arrangeData(data, ress, no_trials=1):
    for mvset, mvsamps in data.items():
        imp_m = []
        imp_f = []
        if no_trials <= 1:
            for mvsam, mvress in mvsamps.items():
                imp_m.append(mvress[0] / mvress[4])
                imp_f.append(mvress[1] / mvress[5])
        else:
            keys = list(mvsamps.keys())
            keys.sort()
            tot_m, tot_f = 0, 0
            for t in range(min(no_trials, len(keys))):
                imp_m += mvsamps[keys[t]][2]
                imp_f += mvsamps[keys[t]][3]
                tot_m = mvsamps[keys[t]][4]
                tot_f = mvsamps[keys[t]][5]
            imp_m = [len(set(imp_m)) / tot_m]
            imp_f = [len(set(imp_f)) / tot_f]
        if mvset not in ress:
            ress[mvset] = [round(np.mean(imp_m)*100,2), round(np.mean(imp_f)*100,2)]
        else:
            ress[mvset] += [round(np.mean(imp_m)*100,2), round(np.mean(imp_f)*100,2)]
    return ress

In [83]:
nets = ['vggvox/v000', 'resnetse34l/v000', 'xvector/v001', 'resnet34vox/v001', 'resnet50vox/v001']

In [84]:
for pol in pols:
    for tar in tars:
        ress = {}
        cols = []
        for net in nets:
            ress = arrangeData(mv_test_results[pol][tar][net], ress, no_trials=1) 
            cols += [net + '-m', net + '-f']
        tar_label = pol.upper() + '  ' + ('FAR@'+str(tar) if tar is not None else 'EER')
        df = pd.DataFrame.from_dict(ress, orient='index', columns=cols)
        df = df.mask(df==0).fillna('-')
        df.sort_index(inplace=True)
        df.columns = pd.MultiIndex.from_tuples([(tar_label, col.split('-')[0], col.split('-')[1]) for col in cols])
        df.style.set_properties(**{'width':'10em', 'text-align':'center'})
        display(HTML(df.to_html()))
        print()

#### Benchmark MV with playback 

In [71]:
mv_test_results_pb = {}
for i1, pol in enumerate(pols):
    mv_test_results_pb[pol] = {}
    for i2, tar in enumerate(tars):
        mv_test_results_pb[pol][tar] = {}
        for i3, net in enumerate(nets):
            mv_test_results_pb[pol][tar][net] = {}
            dp = os.path.join('../data/pt_models', net, 'mvcmp_pb')
            for i4, fp in enumerate(os.listdir(dp)): 
                mvset = fp.split('__')[0]
                mvsam = fp.split('__')[1]
                if int(mvsam.split('.')[0].split('_')[-1]) <=5:
                    if mvset not in mv_test_results_pb[pol][tar][net]:
                        mv_test_results_pb[pol][tar][net][mvset] = {}
                    mv_test_results_pb[pol][tar][net][mvset][mvsam] = computeImpersonation(os.path.join(dp, fp), thrs[None][tar][net], pol) 
                    print('>\r', pol, '(' + str(i1+1) + '/' + str(len(pols)) + ')', tar, '(' + str(i2+1) + '/' + str(len(tars)) + ')', 
                                 net, '(' + str(i3+1) + '/' + str(len(nets)) + ')',  fp, '('+str(i4+1)+'/'+str(len(os.listdir(dp))) +')', end='')
            save_obj(mv_test_results_pb, 'test_vox2_mv_pb_test')
            print(' > saved')

 avg (1/2) None (1/3) resnetse34l/v000 (1/4) gan_mf-f_sv__audio_original_female_5.csv (258/264) > saved
 avg (1/2) None (1/3) xvector/v001 (2/4) real_f-f_mv__mv_01.csv (263/264) > saved
 avg (1/2) None (1/3) resnet34vox/v001 (3/4) real_f-f_mv__mv_01.csv (263/264) > saved
 avg (1/2) None (1/3) resnet50vox/v001 (4/4) gan_mf-f_sv__audio_original_female_5.csv (258/264) > saved
 avg (1/2) 1.0 (2/3) resnetse34l/v000 (1/4) gan_mf-f_sv__audio_original_female_5.csv (258/264) > saved
 avg (1/2) 1.0 (2/3) xvector/v001 (2/4) real_f-f_mv__mv_01.csv (263/264) > saved
 avg (1/2) 1.0 (2/3) resnet34vox/v001 (3/4) real_f-f_mv__mv_01.csv (263/264) > saved
 avg (1/2) 1.0 (2/3) resnet50vox/v001 (4/4) gan_mf-f_sv__audio_original_female_5.csv (258/264) > saved
 avg (1/2) 0.1 (3/3) resnetse34l/v000 (1/4) gan_mf-f_sv__audio_original_female_5.csv (258/264) > saved
 avg (1/2) 0.1 (3/3) xvector/v001 (2/4) real_f-f_mv__mv_01.csv (263/264) > saved
 avg (1/2) 0.1 (3/3) resnet34vox/v001 (3/4) real_f-f_mv__mv_01.csv (

In [85]:
nets = ['resnetse34l/v000', 'xvector/v001', 'resnet34vox/v001', 'resnet50vox/v001']

In [86]:
for pol in pols:
    for tar in tars:
        ress = {}
        cols = []
        for net in nets:
            ress = arrangeData(mv_test_results_pb[pol][tar][net], ress, no_trials=1) 
            cols += [net + '-m', net + '-f']
        tar_label = pol.upper() + '  ' + ('FAR@'+str(tar) if tar is not None else 'EER')
        df = pd.DataFrame.from_dict(ress, orient='index', columns=cols)
        df = df.mask(df==0).fillna('-')
        df.sort_index(inplace=True)
        df.columns = pd.MultiIndex.from_tuples([(tar_label, col.split('-')[0], col.split('-')[1]) for col in cols])
        df.style.set_properties(**{'width':'10em', 'text-align':'center'})
        display(HTML(df.to_html()))
        print()